Lineare Regression
---
Als ersten Anwendungsfall wird ein simples Maschinellen Lernen Verfahren betrachtet, die Lineare Regression. Hierbei wird ein mathematisches Modell durch Trainingsdaten erstellt, welches verwendet werden kann um ein ausgewähltes Feature vorherzusagen. Ein überschaubarer Datensatz für diese Aufgabe wäre zum Beispiel der Iris Datensatz. Dieser beinhaltet Features von drei verschiedenen Blumenarten, welche wiederum durch eine lineare Regression bestimmt werden können.
<br />
Zunächst muss dafür der Datensatz geladen und in die Trainings- und Testmenge aufgeteilt werden. Die beiden Mengen stehen dabei in einem 80% Trainingsdaten - 20% Testdaten Verhältnis.

In [19]:
# Import Dataset from sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

Bibliotheken Beispiel
---
Um einen Eindruck für eine Lineare Regression auf dem Iris Datensatz zu erhalten wird zuerst die Sklearn Implementierung ausprobiert. Die berechneten Abweichungen von den Ergebnissen können wiederum verglichen werden mit weiteren Tests.

In [20]:
lr = LinearRegression()

lr.fit(X_train, y_train)

lr.predict(X_test)
pred = lr.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squared Error:', mean_squared_error(y_test, pred))
print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

Mean Absolute Error: 0.1584601230068247
Mean Squared Error: 0.043573599004634006
Mean Root Squared Error: 0.20874290168682144


From Scratch
---
Da die Implementierung aus Sklearn nicht jeden Datentyp unterstützt muss die Lineare Regression selbst implementiert werden. Der Grund dafür liegt in der homomorphen Verschlüsselung. Sobald eine beliebige Zahl verschlüsselt wurde, ist diese keine gewöhnliche Zahl mehr sonder ein verschlüsseltes Objekt und die meisten Bibliotheken erlauben als Parameter nur Zahlen beziehungsweise primitive Datentypen.
<br />
Für die Lineare Regression werden zwei Funktionen implementiert:
* Fit: In dieser Funktion werden die Gewichte w in der Lineare Regression basierend auf den Trainingsdaten erstellt. Dies geschieht indem immer wieder mit den Gewichten Vorhersagen erstellt werden und verändert werden bis diese konvergieren.
* Predict: In dieser Funktion wird die Vorhersage mit einer beliebigen Eingabe und den in der Predict Funktion berechneten Gewichte erzeugt.

In [21]:
def fit(x, y):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])

	norma = 1
	learning_rate = 0.00001
	epsilon = 0.9
	while(norma > epsilon):
		y_pred = regression @ weights.T
		partial = regression.T @ (y - y_pred)
		norma = np.sum(np.sqrt(np.square(partial)))

		weights = weights.T + (learning_rate * partial)

	return weights

def predict(w, x):
	return w[:-1] @ (np.array(x).T) + w[-1]


Nachdem die Funktionen implementiert sind können die Gewichte erstellt werden mit den Trainingsdaten. Sobald diese berechnet sind wird ein Feature von den Testdaten vorhersagt. Dabei wird noch die Laufzeit der Vorhersage gemessen um später mit den anderen Tests verglichen zu werden.
<br />
Die erstellten Gewichte können in den Tests mit der homomorphen Verschlüsselung wieder verwendet werden.

In [22]:
weights = fit(X_train, y_train)

s_predict = datetime.datetime.now()
pred = predict(weights, X_test)
e_predict = datetime.datetime.now()

def meanError(y_test, pred):
	print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
	print('Mean Squared Error:', mean_squared_error(y_test, pred))
	print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

meanError(y_test, pred)

print(f"Prediction time for original: {int((e_predict-s_predict).total_seconds() * 1000)}ms")

def timer(encryptTime, predictTime, decryptTime):
	print(f"Encryption time: {int(encryptTime.total_seconds() * 1000)}ms")
	print(f"Prediction time: {int(predictTime.total_seconds() * 1000)}ms")
	print(f"Decryption time: {int(decryptTime.total_seconds() * 1000)}ms")


Mean Absolute Error: 0.15791782248674335
Mean Squared Error: 0.042219357807520355
Mean Root Squared Error: 0.2054734966060595
Prediction time for original: 0ms


Die berechneten Abweichung zwischen den Vorhersagen und den tatsächlichen Features sind ähnlich zu den Ergebnissen der Sklearn Implementierung und weisen erst ab der 3. Nachkommastelle Unterschiede auf. Die eigene Implementierung scheint dadurch korrekt zu funktionieren.

Vorhersagen auf verschlüsselten Daten
===
In den folgenden zwei Beispielen werden die mit unverschlüsselten Daten trainierten Gewichte angewendet auf verschlüsselte Daten.

Verschlüsselung mit Pyfhel
---

In [23]:
from Pyfhel import Pyfhel

pyfhel = Pyfhel()
pyfhel.contextGen(p=65537)

pyfhel.keyGen()

def encrypt(data):
    return [[pyfhel.encryptFrac(feature) for feature in item] for item in data]

def decrypt(data):
    return [pyfhel.decryptFrac(item) for item in data]

Für diese Bibliothek muss die predict Funktion etwas umgeschrieben werden, da die Matrizenmultiplikation Operation nicht funktioniert zwischen Skalaren und verschlüsselten Objekten.

In [24]:
def predictPyfhel(w, x):
    transX = np.array(x).T
    result = []
    for i in range(transX.shape[1]):
        for j in range(transX.shape[0]):
            scaledFeature = transX[j][i] * w[j]
            if len(result) == i:
                result.append(scaledFeature)
            else:
                result[i] = result[i] + scaledFeature
    return np.array(result) + w[-1]

Als letzter Schritt werden alle Testdaten verschlüsselt, die Vorhersage erzeugt und das Ergebnis entschlüsselt.

In [25]:
s_encrypt = datetime.datetime.now()
eX_test = encrypt(X_test)
e_encrypt = datetime.datetime.now()

s_predict = datetime.datetime.now()
ePred = predictPyfhel(weights, eX_test)
e_predict = datetime.datetime.now()

s_decrypt = datetime.datetime.now()
cryptPred = decrypt(ePred)
e_decrypt = datetime.datetime.now()

meanError(y_test, cryptPred)
timer(e_encrypt-s_encrypt, e_predict-s_predict, e_decrypt-s_decrypt)

Mean Absolute Error: 0.1579178223464017
Mean Squared Error: 0.04221935761486088
Mean Root Squared Error: 0.20547349613724122
Encryption time: 125ms
Prediction time: 18ms
Decryption time: 3ms


* Funktionalität: Ergebnisse sind fast identisch zum Original (Abweichungen in der Gleitkommarepräsentation)
* Laufzeit: Fast gleich schnell wie die Implementierung

Verschlüsselung mit Paillier
---

In [26]:
from phe import paillier

public_key, private_key = paillier.generate_paillier_keypair()

def encrypt(data):
    return [[public_key.encrypt(feature) for feature in item] for item in data]

def decrypt(data):
    return [private_key.decrypt(item) for item in data]

Für die Paillier Bibliothek in Python muss keine gesonderte predict Funktion implementiert werden, da die Matrizenmultiplikation möglich ist. Dadurch müssen nur noch die Daten verschlüsselt, die Vorhersage erzeugt und das Ergebnis entschlüsselt werden.

In [27]:
s_encrypt = datetime.datetime.now()
eX_test = encrypt(X_test)
e_encrypt = datetime.datetime.now()

s_predict = datetime.datetime.now()
ePred = predict(weights, eX_test)
e_predict = datetime.datetime.now()

s_decrypt = datetime.datetime.now()
cryptPred = decrypt(ePred)
e_decrypt = datetime.datetime.now()

meanError(y_test, cryptPred)
timer(e_encrypt-s_encrypt, e_predict-s_predict, e_decrypt-s_decrypt)

Mean Absolute Error: 0.15791782248674335
Mean Squared Error: 0.042219357807520355
Mean Root Squared Error: 0.2054734966060595
Encryption time: 9626ms
Prediction time: 490ms
Decryption time: 692ms


* Funktionalität: Identisches Ergebnis zu der unverschlüsselten Version
* Laufzeit: Benötigt deutlich länger